## Here's the API

http://docs.openmm.org/latest/api-python/index.html

## Here's the documentation

http://docs.openmm.org/latest/userguide/index.html

## Here's an example

In [1]:
from simtk import openmm
from simtk.openmm import app, unit
from openmmforcefields.generators import SystemGenerator
import openmmtools

In [2]:
# Load PDB
pdb = app.PDBFile("../../input/ala_vacuum_noh.pdb")

In [3]:
# Set up modeller
modeller = app.Modeller(pdb.topology, pdb.positions)

In [4]:
# Create Forcefield object
ffxmls = ["amber14/protein.ff14SB.xml", "amber14/tip3p.xml"]
forcefield = app.ForceField(*ffxmls)



In [5]:
# Add hydrogens
modeller.addHydrogens(forcefield, pH=7.0)

[None, None, None]

In [6]:
# Add solvent
modeller.addSolvent(forcefield, ionicStrength=150 * unit.millimolar, padding=1.0 * unit.nanometers)

In [7]:
# Save solvated PDB
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("solvated.pdb", 'w'), keepIds=True)


In [8]:
# Generate system and add barostat
temperature = 300 * unit.kelvin
system = forcefield.createSystem(modeller.topology, 
                                 nonbondedMethod=app.PME,
                                 nonbondedCutoff=1 * unit.nanometer, 
                                 removeCMMotion=False,
                                 constraints=app.HBonds, 
                                 hydrogenMass=4 * unit.amu)

barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50)
system.addForce(barostat)

# If you already have prmtop and inpcrd files generated by tleap, you can load those in using:
# http://docs.openmm.org/latest/api-python/generated/openmm.app.amberprmtopfile.AmberPrmtopFile.html#openmm.app.amberprmtopfile.AmberPrmtopFile
# http://docs.openmm.org/latest/api-python/generated/openmm.app.amberinpcrdfile.AmberInpcrdFile.html#openmm.app.amberinpcrdfile.AmberInpcrdFile
# and then call createSystem() (like above) on your loaded AmberPrmtopFile object.

4

In [9]:
# Serialize the system to an xml file. This is will be necessary for resuming a simulation
with open('system.xml', 'w') as f:
    f.write(openmm.XmlSerializer.serialize(system))

In [10]:
# Set up integrator
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)

# Note: we use the the openmmtools LangevinIntegrator rather than the OpenMM one because it is uses
# BAOAB, wheras the OpenMM doesn't. Here are the OpenMM integrators: 
# http://docs.openmm.org/latest/api-python/library.html#integrators

In [11]:
# Configure platform
platform = openmm.Platform.getPlatformByName('CUDA')
platform.setPropertyDefaultValue('Precision', 'mixed')

In [12]:
# Set up simulation and minimize it
simulation = app.Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
simulation.context.setVelocitiesToTemperature(temperature)
simulation.minimizeEnergy()

In [13]:
# Save energy minimized PDB
positions_minimized = simulation.context.getState(getPositions=True, enforcePeriodicBox=False).getPositions()
app.PDBFile.writeFile(modeller.topology, positions_minimized, open("minimized.pdb", 'w'), keepIds=True)


In [14]:
# Set up reporters for state data, checkpoint file, and trajectory, if desired
simulation.reporters.append(app.StateDataReporter('ala.csv', 12500, time=True, step=True,
        potentialEnergy=True, kineticEnergy=True, totalEnergy=True, volume=True, temperature=True))
simulation.reporters.append(app.CheckpointReporter('ala.chk', 12500))
simulation.reporters.append(app.DCDReporter('ala.dcd', 12500, enforcePeriodicBox=False))


In [15]:
# Run the simulation
n_steps = 100
n_iterations = 10
for _ in range(n_iterations):
    simulation.step(n_steps)
    
    # Write the state every 100 steps. This is will be necessary for resuming a simulation
    state = simulation.context.getState(getPositions=True, getVelocities=True)
    with open('state.xml', 'w') as outfile:
        state_xml = openmm.XmlSerializer.serialize(state)
        outfile.write(state_xml)


## Example for resuming a simulation

In [16]:
from simtk import openmm
from simtk.openmm import app
import openmmtools

In [18]:
# Read in solvated PDB
pdb = app.PDBFile("solvated.pdb")

# Load system
with open('system.xml', 'r') as f:
    system = openmm.XmlSerializer.deserialize(f.read())

# Load state
with open('state.xml', 'r') as infile:
    state_xml = infile.read()
    state = openmm.XmlSerializer.deserialize(state_xml)


In [19]:
# Set up integrator
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)


In [20]:
# Set up platform
platform = openmm.Platform.getPlatformByName('CUDA')
platform.setPropertyDefaultValue('Precision', 'mixed')

In [21]:
# Set up simulation 
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
simulation.context.setPositions(state.getPositions())
simulation.context.setVelocities(state.getVelocities())
simulation.context.setTime(state.getTime())

... Everything else is the same as it was in the previous example